In [2]:
using Printf
using ArgParse
using Logging

using Dates
using CSV
using NPZ
using JSON
using HDF5
using DataStructures
using DelimitedFiles
using DataFrames
using NetCDF


base_folder = ".."

include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac_aux.jl"))
include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac_io.jl"))
include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac.jl"))

run_epidemic_spreading_mmca!

In [3]:
data_folder  = "../data/"
config_fname = "../test/config_test.json"
output_folder = "./"
seeds_fname   = "../data/A0_initial_conditions.csv"

config          = JSON.parsefile(config_fname);
data_dict       = config["data"]
pop_params_dict = config["population_params"]
epi_params_dict = config["epidemic_params"]

output_format = "netcdf"

"netcdf"

In [8]:
# Reading metapopulation Dataframe
metapop_data_filename = joinpath(data_folder, data_dict["metapopulation_data_filename"])
metapop_df            = CSV.read(metapop_data_filename, DataFrame)

# Loading metapopulation patches info (surface, label, population by age)
metapop_data_filename = joinpath(data_folder, data_dict["metapopulation_data_filename"])
metapop_df = CSV.read(metapop_data_filename, DataFrame)

# Loading mobility network
mobility_matrix_filename = joinpath(data_folder, data_dict["mobility_matrix_filename"])
network_df  = CSV.read(mobility_matrix_filename, DataFrame)

# Metapopulations patches coordinates (labels)
M_coords = map(String,metapop_df[:, "id"])
M = length(M_coords)

# Coordinates for each age strata (labels)
G_coords = map(String, pop_params_dict["age_labels"])
G = length(G_coords)

T = 1

# Num. of vaccination statuses Vaccinated/Non-vaccinated
population = init_pop_param_struct(G, M, G_coords, pop_params_dict, metapop_df, network_df)
epi_params = init_epi_parameters_struct(G, M, T, G_coords, epi_params_dict)

V = epi_params.V
N = epi_params.NumComps
comp_coords = epi_params.CompLabels
V_coords = ["NV", "V"]

2-element Vector{String}:
 "NV"
 "V"

In [44]:
# Loading initial 
conditions₀ = CSV.read(seeds_fname, DataFrame);

M_idx  = Int.(conditions₀[:,"idx"])
conditions₀ = Int64.(round.(conditions₀[:,"seed"]))

seeds_age_ratio = pop_params_dict["seeds_age_ratio"]

S₀_idx = 1
A₀_idx = 3
V_idx  = 1

A₀ = zeros(Float64, G, M);
A₀[:, M_idx] .= seeds_age_ratio .* transpose(conditions₀)
S₀ = population.nᵢᵍ .- A₀

compartments = zeros(Float64, G, M, V, N);
compartments[:, :, V_idx, S₀_idx] .= S₀
compartments[:, :, V_idx, A₀_idx] .= A₀

3×2850 view(::Array{Float64, 4}, :, :, 1, 3) with eltype Float64:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [45]:
output_fname = joinpath(output_folder, "initial_condition.nc")

isfile(output_fname) && rm(output_fname)
if output_format == "netcdf"
    nccreate(output_fname, "data", "G", G_coords, "M", M_coords, "V", V_coords, "epi_states", comp_coords)
    ncwrite(compartments, output_fname, "data")
elseif output_format == "netcdf"
    output_fname = joinpath(output_folder, "initial_condition.h5")
    isfile(output_fname) && rm(output_fname)
    h5open(output_fname, "w") do file
        write(file, "data", compartments)
    end
end
    

LoadError: syntax: incomplete: premature end of input

In [61]:
T = 100

# Num. of vaccination statuses Vaccinated/Non-vaccinated
population = init_pop_param_struct(G, M, G_coords, pop_params_dict, metapop_df, network_df)
epi_params = init_epi_parameters_struct(G, M, T, G_coords, epi_params_dict)

t₀ = 1

for i in 1:2
    epi_params.ρˢᵍᵥ[:,:,t₀,i]  .= compartments[:, :, i, 1] ./ population.nᵢᵍ
    epi_params.ρᴱᵍᵥ[:,:,t₀,i]  .= compartments[:, :, i, 2] ./ population.nᵢᵍ
    epi_params.ρᴬᵍᵥ[:,:,t₀,i]  .= compartments[:, :, i, 3] ./ population.nᵢᵍ
    epi_params.ρᴵᵍᵥ[:,:,t₀,i]  .= compartments[:, :, i, 4] ./ population.nᵢᵍ
    epi_params.ρᴾᴴᵍᵥ[:,:,t₀,i] .= compartments[:, :, i, 5] ./ population.nᵢᵍ
    epi_params.ρᴾᴰᵍᵥ[:,:,t₀,i] .= compartments[:, :, i, 6] ./ population.nᵢᵍ
    epi_params.ρᴴᴿᵍᵥ[:,:,t₀,i] .= compartments[:, :, i, 7] ./ population.nᵢᵍ
    epi_params.ρᴴᴰᵍᵥ[:,:,t₀,i] .= compartments[:, :, i, 8] ./ population.nᵢᵍ
    epi_params.ρᴿᵍᵥ[:,:,t₀,i]  .= compartments[:, :, i, 9] ./ population.nᵢᵍ
    epi_params.ρᴰᵍᵥ[:,:,t₀,i]  .= compartments[:, :, i, 10] ./ population.nᵢᵍ
end

epi_params.ρˢᵍᵥ[isnan.(epi_params.ρˢᵍᵥ)]   .= 0
epi_params.ρᴱᵍᵥ[isnan.(epi_params.ρᴱᵍᵥ)]   .= 0
epi_params.ρᴬᵍᵥ[isnan.(epi_params.ρᴬᵍᵥ)]   .= 0
epi_params.ρᴵᵍᵥ[isnan.(epi_params.ρᴵᵍᵥ)]   .= 0
epi_params.ρᴾᴴᵍᵥ[isnan.(epi_params.ρᴾᴴᵍᵥ)] .= 0
epi_params.ρᴾᴰᵍᵥ[isnan.(epi_params.ρᴾᴰᵍᵥ)] .= 0
epi_params.ρᴴᴿᵍᵥ[isnan.(epi_params.ρᴴᴿᵍᵥ)] .= 0
epi_params.ρᴴᴰᵍᵥ[isnan.(epi_params.ρᴴᴰᵍᵥ)] .= 0
epi_params.ρᴿᵍᵥ[isnan.(epi_params.ρᴿᵍᵥ)]   .= 0
epi_params.ρᴰᵍᵥ[isnan.(epi_params.ρᴰᵍᵥ)]   .= 0

sum(epi_params.ρᴬᵍᵥ[3, :, 1, 1] .* population.nᵢᵍ[1,:])

128.07964628584256